# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [4]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [177]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-03_Leg-2/'

In [178]:
cruises = os.listdir(basepath+array)

In [179]:
cruises

['Pioneer-01',
 'Pioneer-02',
 'Pioneer-02a',
 'Pioneer-03a',
 'Pioneer-03_Leg-1',
 'Pioneer-03_Leg-2',
 'Pioneer-04',
 'Pioneer-04a',
 'Pioneer-05',
 'Pioneer-05a',
 'Pioneer-06',
 'Pioneer-06a',
 'Pioneer-07',
 'Pioneer-07a',
 'Pioneer-07b',
 'Pioneer-08',
 'Pioneer-08a',
 'Pioneer-09',
 'Pioneer-09a',
 'Pioneer-10',
 'Pioneer-10a',
 'Pioneer-10b',
 'Pioneer-10c',
 'Pioneer-11',
 'Pioneer-11a',
 'Pioneer-11b']

In [180]:
os.listdir(basepath+array+cruise)

['cruise.metadata', 'CRUISE_ID', 'ctd', 'Water Sampling']

In [185]:
os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/')

['0001A001.SAL',
 '0001A001SAL.csv',
 '0002A001.SAL',
 '0002A001SAL.csv',
 '0004A001.SAL',
 '0004A001SAL.csv',
 '001OXY.xlsx',
 '002OXY.xlsx',
 '004OXY.xlsx']

In [182]:
sal_header = ['cruise', 'station #', 'cast #', 'salinometer #', 'standard seawater batch', 'salinometer bath temperature', 'room temperature', 'case', 'salinometer standby #', 'operator', 'date and time']

In [183]:
for files in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'):
    if '.SAL' in files:
        header = {}
        sample = []
        salinity = []
        with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+files) as file:
            data = file.readlines()
            for line in data:
                if len(header) == 0:
                    strs = data[0].replace('"','').split(',')
                    strs = [x for x in strs if len(x) > 0] 
                    for ind,hdr in enumerate(sal_header):
                        header.update({hdr:strs[ind]})
                elif int(line.split()[0]) == 0:
                    pass
                else:
                    strs = line.split()
                    sample.append(strs[0])
                    salinity.append(strs[2])
            # Generate a pandas dataframe to populate data
            data_dict = {'cruise':header['cruise'],'station #':header['station #'],'cast #':header['cast #'],
                         'case':header['case'],'sample':sample,'salinity':salinity}
            df = pd.DataFrame.from_dict(data_dict)
            df.to_csv(file.name.replace('.','')+'.csv')

In [184]:
file.name

'/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-03_Leg-2/Water Sampling/Salts and O2/0004A001.SAL'

In [175]:
df

,cruise,station #,cast #,case,sample,salinity
0,KN222,015,1,J,1,35.6530
1,KN222,015,1,J,2,35.6528
2,KN222,015,1,J,3,35.1699
3,KN222,015,1,J,4,35.1902
4,KN222,015,1,J,5,35.8753
5,KN222,015,1,J,6,35.8823
6,KN222,015,1,J,7,35.7738
7,KN222,015,1,J,8,35.7738


In [136]:
header

{'cruise': 'SHORE',
 'station #': '002',
 'cast #': '1',
 'salinometer #': '12',
 'standard seawater batch': 'P-141',
 'salinometer bath temperature': '24',
 'room temperature': '21',
 'case': 'S',
 'salinometer standby #': '1.268132',
 'operator': 'DW',
 'date and time': '11-23-2013 15:38\n'}

['35.6323', '35.6338', '33.5974', '33.6013']

In [104]:
data

    

['"SHORE","002","1","12","P-141","","24","21","S","1.268132","DW","11-23-2013 15:38"\n',
 '   1          2.03208            35.6323      2 \n',
 '   2          2.03216            35.6338      2 \n',
 '   3          1.92850            33.5974      2 \n',
 '   4          1.92870            33.6013      2 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000            -9.0000      9 \n',
 '   0          0.00000        

In [103]:
header

{'cruise': 'SHORE',
 'station #': '002',
 'cast #': '1',
 'salinometer #': '12',
 'standard seawater batch': 'P-141',
 'salinometer bath temperature': '24',
 'room temperature': '21',
 'case': 'S',
 'salinometer standby #': '1.268132',
 'operator': 'DW',
 'date and time': '11-23-2013 15:38\n'}

In [98]:
len(sal_header)

11

In [60]:
for files in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'):
    if '.SAL' in files:
        sample = []
        unknown = []
        salinity = []
        with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+files) as file:
            for line in file.readlines():
                line = line.replace(',',' ')
                strs = line.split()
                if line.startswith('"'):
                    for x in strs:
                        
                    cast = strs[1].replace('"','')
                    case = strs[8].replace('"','')
                else:
                    sample.append(strs[0])
                    unknown.append(strs[1])
                    salinity.append(strs[2])
        data = {'Cast #':cast,'Sample #':sample,'Case':case,'Unknown':unknown,'Salinity':salinity}
        df = pd.DataFrame(data=data)
        df = df[df['Salinity'] != '-9.0000']
        df.to_excel(file.name.replace('.','')+'.xlsx')
            

In [63]:
file.name

'/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-01/Water Sampling/Salts and O2/0007A001.SAL'

'001.SAL'

In [65]:
pd.read_excel(basepath+array+cruise+'Water Sampling/Salts and O2/0003A001SAL.xlsx')

,Cast #,Sample #,Case,Unknown,Salinity
0,3,1,S,2.03266,35.6437
1,3,2,S,2.02930,35.5774
2,3,3,S,1.93460,33.7167
3,3,4,S,1.93118,33.6498


In [66]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+'KN-214_CTD/'
summary_sheet_path = basepath+array+cruise+'/Water Sampling/Pioneer-1_KN-214_Sampling_Log.xlsx'
salts_and_o2_path = basepath+array+cruise+'/Water Sampling/Irminger_Sea-02_AT-30-01_Oxygen_and_Salnity_Sample_Data/'
nutrients_path = basepath+array+cruise+'/Water Sampling/Irminger_Sea-02_AT-30-01_Nutrients_Sample_Data_2016-09-01_ver_1-00.xlsx'

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df.columns.values

In [ ]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

In [ ]:
#df.to_csv(dirpath+'Irminger-5_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
# Need a way to map the oxygen and salinity samples to the CTD samples
# Load the CTD sample logs

In [67]:
sample_log = pd.read_excel(summary_sheet_path,sheet_name='Summary',header=0)

In [68]:
sample_log.head()

,Station-Cast #,Niskin #,Rosette Position,Date,Time,Trip Depth,Oxygen Bottle #,Ph Bottle #,DIC/TA Bottle #,Duplicate Ph Bottle #,...,Salts Bottle #,Nitrate Bottle 1,Nitrate Bottle 2,Duplicate Nitrate Bottle 1,Duplicate Nitrate Bottle 2,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample #,Duplicate Chlorophyll Brown Bottle #,Duplicate Chlorophyll Filter Sample #,Comments
0,2,1,1,2013-11-21,NaN,125 m,V 1,002,003,–,...,S 1,2-1,2-2,–,–,1,002 / 125 / 1,2,002 / 125 / 2,NaN
1,2,2,2,2013-11-21,NaN,125 m,–,–,–,–,...,–,–,–,–,–,–,–,–,–,No water samples from this bottle
2,2,3,3,2013-11-21,NaN,111.5 m,V 2,004,005,–,...,S 2,2-3,2-4,–,–,3,002 / 112 / 3,4,002 / 112 / 4,NaN
3,2,4,4,2013-11-21,NaN,111.5 m,–,–,–,–,...,–,–,–,–,–,–,–,–,–,No water samples from this bottle
4,2,5,5,2013-11-21,NaN,38.1 m,V 3,006,007,–,...,S 3,2-5,2-6,–,–,5,002 / 38 / 5,6,002 / 38 / 6,NaN


In [69]:
# Rename the column headers
sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: int(x.replace('*','')) if type(x) == str else x )
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: x if np.isnan(x) else str( int(x) ) )
sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Station-Cast #'].apply(lambda x: str(x).zfill(3))

In [70]:
set(list(sample_log['Sample Log: Niskin #']))

{'1', '2', '3', '4', '5', '6', '7', '8'}

In [71]:
# Load the oxygen data
sal_dir = salts_and_o2_path
sal_df = pd.DataFrame()
oxy_df = pd.DataFrame()

for file in os.listdir(sal_dir):
    if 'SAL' in file:
        sal_df = sal_df.append(pd.read_excel(sal_dir+file))
    elif 'OXY' in file:
        oxy_df = oxy_df.append(pd.read_excel(sal_dir+file))

FileNotFoundError: [Errno 2] No such file or directory: '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-01//Water Sampling/Irminger_Sea-02_AT-30-01_Oxygen_and_Salnity_Sample_Data/'

In [ ]:
oxy_df

In [ ]:
sal_df.rename(columns=lambda x: 'SAL: ' + x.strip(), inplace=True)
sal_df.dropna(subset=['SAL: Cruise'], inplace=True)
sal_df.fillna(-999, inplace=True)
sal_df['SAL: Station'] = sal_df['SAL: Station'].apply(lambda x: str( int(x) ).zfill(3) )
sal_df['SAL: Niskin #'] = sal_df['SAL: Niskin #'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Sample'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Case ID'] + sal_df['SAL: Sample']

In [ ]:
oxy_df.rename(columns=lambda x: 'OXY: ' + x.strip(), inplace=True)
oxy_df.dropna(subset=['OXY: Cruise'], inplace=True)
oxy_df.fillna(-999, inplace=True)
oxy_df['OXY: Station'] = oxy_df['OXY: Station'].apply(lambda x: str( int(x) ).zfill(3) )
oxy_df['OXY: Niskin #'] = oxy_df['OXY: Niskin #'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Sample#'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Case'] + oxy_df['OXY: Sample#']

In [ ]:
# Now I need to merge dataframes together
sample_log = sample_log.merge(sal_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Salts Bottle #'], right_on=['SAL: Station','SAL: Sample'])

In [ ]:
sample_log = sample_log.merge(oxy_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Oxygen Bottle #'],
                              right_on=['OXY: Station','OXY: Sample#'])

In [ ]:
os.listdir('Irminger/Irminger-5/Water Sampling/')

In [ ]:
nutrients = pd.read_excel(nutrients_path,header=0)

In [ ]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split(',').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [ ]:
nutrient_bottles

In [ ]:
# Add the nutrient bottle number back into the sample log, and remove the excess '.'
sample_log = sample_log.join(nutrient_bottles)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace('.','') if type(x) == str else x)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace(' ','') if type(x) == str else x)

In [ ]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log = sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

In [ ]:
sample_log.columns.values

In [ ]:
sample_columns = ['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy','Avg: Nitrate+Nitrite [µmol/L]', 'Avg: Ammonium [µmol/L]',
    'Avg: Phosphate [µmol/L]', 'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]', 'Avg: Nitrate [µmol/L]']

In [ ]:
result = df.merge(sample_log[sample_columns], how='left',
                  left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [ ]:
result

In [ ]:
result.columns.values

In [ ]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

In [ ]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [ ]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('Avg:','Bottle') if 'Avg:' in x else x, inplace=True)

In [ ]:
result

In [ ]:
# Replace all of the nans with -999999 and save to a csv
result.fillna(str(-9999999),inplace=True)

In [ ]:
result

In [ ]:
result.to_csv(salts_and_o2_path+'Irminger-3_Summary.csv')

In [ ]:
os.getcwd()